In [149]:
from kaggle_environments import evaluate, make, utils
import os
import sys
import inspect
from pyplAI_algorithms import minmax_agent, mcts_agent

env = make("connectx", debug=True)
print(env.render(mode="ansi"))

+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+
| 0 | 0 | 0 | 0 | 0 | 0 | 0 |
+---+---+---+---+---+---+---+



In [150]:
# Random agent
def random_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

In [151]:
# Evaluating agent against random and negamax
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / len(rewards)

def evaluate_agent(agent, num_episodes=10):
    rewards_random = evaluate("connectx", [agent, "random"], num_episodes=num_episodes)
    rewards_negamax = evaluate("connectx", [agent, "negamax"], num_episodes=num_episodes)
    
    print("My Agent vs Random Agent:", mean_reward(rewards_random))
    print("My Agent vs Negamax Agent:", mean_reward(rewards_negamax))  

In [152]:
# Debugging agent against random
def debug_agent(agent):
    env = make("connectx", debug=True)
    trainer = env.train([None, "random"])
    observation = trainer.reset()

    while not env.done:
        my_action = agent(observation, env.configuration)
        print(f"Turn {observation.step + 1}: Action {my_action}")
        observation, reward, done, info = trainer.step(my_action)   

    print(f"\nYou won :)\n" if env.state[0].reward > 0 else "\nYou lost :(\n")
    print(env.render(mode="ansi"))

In [153]:
# Facing agents
def facing_agents(agent_a, agent_b, num_episodes=10):
    results = []

    env = make("connectx", debug=True)

    for i in range(num_episodes):
        env.reset()
        env.run([agent_a, agent_b])
        results.append(env.state[0].reward)

    print(f"Agent A won {results.count(1)} times")
    print(f"Agent B won {results.count(-1)} times")
    print(f"Draw {results.count(0)} times")

In [154]:
# Writing agent to file and validating submission format
def write_agent_to_file(agent):
    file = agent.__name__ + ".py"
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(agent))

def validate_submission(agent):
    out = sys.stdout
    agent = utils.read_file(agent.__name__ + ".py")
    sys.stdout = out

    env = make("connectx", debug=True)
    env.run([agent, agent])

    print("Success :)" if env.state[0].status == env.state[1].status == "DONE" else "Failed :(")